<h1>Hip roof</h1>

<h2>Simple reference model</h2>
![alt text](reference_model_simplex.gif)

In [346]:
from pyplasm import *
def roof_simple():
    verts = [[0,0,0],[20,0,0],[20,10,0],[0,10,0],[5,5,3],[15,5,3],[5,5,0],[15,5,0]]
    cells = [[1,4,5,7],[1,2,5,6,7,8],[2,6,3,8],[3,4,5,6,7,8]]
    roof = MKPOL([verts, cells, None])
    return roof

In [347]:
roof_simple = roof_simple()
skeleton_roof_simple = SKEL_1(roof_simple)
VIEW(OFFSET([.2,.2,.4])(skeleton_roof_simple))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000AA28960> >

<h2>Final result (simple)</h2>
![alt text](skeleton_simple.jpg)
![alt text](hip_roof_simple.jpg)

<h2>Complex reference model</h2>
![alt text](reference_model_complex.jpg)

In [342]:
def roof_complex():
    verts = [[0,10,0],[12,10,0],[12,0,0],[20,0,0],[16,4,0],[16,4,3],[16,15,0],[16,15,3],[20,20,0],[0,20,0],[5,15,0],[5,15,3]]
    cells = [[1,2,7,8,11,12],[1,11,12,10],[10,11,12,7,8,9],[2,3,5,6,7,8],[4,5,6,7,8,9],[3,4,6]]
    roof = MKPOL([verts, cells, None])
    return roof

In [343]:
roof_complex = roof_complex()
skeleton_roof_complex = SKEL_1(roof_complex)
verts=UKPOL(skeleton_roof_complex)
newVerts=[]
bassi=[]
newSkeleton=[]

for i in range(0,len(verts[0])):
    elem=verts[0][i]
    if elem[0]>=0 and elem[1]>=0 and elem[2]==0:
        newVerts.append(elem)
    if elem[0]>=0 and elem[1]>=0 and elem[2]>0:
        newVerts.append(elem)
        x=elem[0]
        y=elem[1]
        bassi.append([x,y])
for j in range(0,len(newVerts)):
    elem2=newVerts[j]
    x=elem2[0]
    y=elem2[1]
    elemBasso=[x,y]
    if elem2[2]==0 and elemBasso not in bassi and elem2 not in newSkeleton:
        newSkeleton.append(elem2)
    elif elem2[2]!=0 and elem2 not in newSkeleton:
        newSkeleton.append(elem2)

print(newSkeleton)
VIEW(OFFSET([.2,.2,.4])(skeleton_roof_complex))

[[12.0, 10.0, 0.0], [16.0, 15.0, 3.0], [12.0, 10.000000953674316, 0.0], [0.0, 10.0, 0.0], [9.5367431640625e-07, 10.0, 0.0], [5.0, 15.0, 3.0], [0.0, 20.0, 0.0], [4.999999046325684, 15.0, 0.0], [20.0, 20.0, 0.0], [12.0, 4.76837158203125e-07, 0.0], [16.0, 4.0, 3.0], [12.0, 9.5367431640625e-07, 0.0], [12.0, 0.0, 0.0], [12.000000953674316, 10.0, 0.0], [20.0, 0.0, 0.0], [20.0, 9.5367431640625e-07, 0.0], [20.0, 7.152557373046875e-07, 7.152557373046875e-07], [16.0, 3.9999992847442627, 3.0], [12.0, 7.152557373046875e-07, 7.152557373046875e-07]]


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000AA28990> >

<h2>Final result (complex)</h2>
![alt text](skeleton_complex.jpg)
![alt text](hip_roof_complex.jpg)